In [1]:
import pandas as pd
import datetime
import numpy as np
import os
from string_grouper import match_strings

In [2]:
rawpatents = pd.read_csv("C:\\Users\\fjying\\Desktop\\RA_Columbia\\patentparsing\\identifycompanypermno\\input\\rawpatentrecord.csv")

C:\Users\fjying\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
os.chdir("C:\\Users\\fjying\\Desktop\\RA_Columbia\\patentparsing\\identifycompanypermno\\intermediate_output")

In [4]:
firstpatentassignments = pd.read_csv("first_patent_assignment_1980_2015_standardized.csv")

In [5]:
firstpatentassignments = firstpatentassignments[~firstpatentassignments['rawname_0'].isna()]

In [6]:
rawpatents = rawpatents.dropna(subset = ['patnum', 'permno'])
rawpatents = rawpatents.drop_duplicates(['patnum', 'permno'])
rawpatents = rawpatents.rename(columns = {'patnum': 'patent_ids'})

In [7]:
rawpatents['patent_ids'] = rawpatents['patent_ids'].astype(str)
firstpatentassignments['patent_ids'] = firstpatentassignments['patent_ids'].astype(str)

In [8]:
duplicatedpatentids = np.unique(rawpatents['patent_ids'][rawpatents['patent_ids'].duplicated()].values)

In [9]:
rawpatents_noduplicated = rawpatents[~rawpatents['patent_ids'].isin(duplicatedpatentids)]
rawpatents_duplicated = rawpatents[rawpatents['patent_ids'].isin(duplicatedpatentids)]

In [10]:
crosswalk = pd.merge(rawpatents_noduplicated, firstpatentassignments, how= 'inner')

In [11]:
crosswalk = crosswalk[['name_0', 'permno', 'recorded_date', 'patent_ids']]

In [12]:
crosswalk = crosswalk.sort_values(['recorded_date'])

In [13]:
crosswalk = crosswalk.drop_duplicates(['name_0', 'permno'])

In [14]:
#Resolve relationship between one permno and multiple company names
duplicated_permnos = np.unique(crosswalk['permno'][crosswalk['permno'].duplicated()].values)

In [15]:
crosswalk_one_on = crosswalk[~crosswalk['permno'].isin(duplicated_permnos)]
crosswalk_multiple_on_one = crosswalk[crosswalk['permno'].isin(duplicated_permnos)]

In [16]:
crosswalk_final = pd.DataFrame()
for i in duplicated_permnos:
    temp = crosswalk[crosswalk['permno'] == i]
    temp = temp.drop_duplicates(subset = ['name_0'], keep = 'first')
    try:
        matches = match_strings(temp['name_0'])
    except ValueError:
        print("only stop words")
        continue
    matches = matches[matches['left_side'] != matches['right_side']]
    if matches.empty:
        continue
    name = matches['left_side'].value_counts().keys()[0]
    crosswalk_final = crosswalk_final.append(temp[temp['name_0'] == name])

only stop words


In [17]:
crosswalk_final = crosswalk_final.append(crosswalk_one_on)

In [18]:
#resolve one company name multiple permnos due to incomplete patent records
crosswalk_final_one_comp_multiple_permno = crosswalk_final[crosswalk_final['name_0'].duplicated(keep = False)].sort_values(['name_0'])

In [19]:
crosswalk_final_one_on_one = crosswalk_final[~crosswalk_final['name_0'].duplicated(keep = False)]

In [20]:
crosswalk_final_one_comp_multiple_permno.to_csv("crosswalk_final_one_comp_multiple_permno.csv", index = False)

In [21]:
crsp_directory = pd.read_csv("C:\\Users\\fjying\\Desktop\\RA_Columbia\\patentparsing\\identifycompanypermno\\input\\crsp_directory.csv")

In [22]:
for i in crosswalk_final_one_comp_multiple_permno['permno'].values:
    print(i)
    display(crsp_directory[crsp_directory['PERMNO'] == i]['NAME OF FIRM'].values)

43342.0


array(['E SYSTEMS INC', 'L T V ELECTROSYSTEMS INC'], dtype=object)

79472.0


array(['AETRIUM INC'], dtype=object)

78950.0


...

array(['TIVOLI INDUSTRIES INC'], dtype=object)

85908.0


array(['GRACE W R & CO DEL NEW'], dtype=object)

25005.0


array(['GRACE W R & CO'], dtype=object)

In [23]:
temp = pd.read_csv("crosswalk_final_one_comp_multiple_permno_resolved.csv")
crosswalk_final = crosswalk_final_one_on_one.append(temp)

In [24]:
crosswalk_final = crosswalk_final.drop_duplicates(['name_0', 'permno'])

In [25]:
#link first patent assignments with raw patent records with duplicated patent ids

In [26]:
crosswalk_2 = pd.merge(rawpatents_duplicated, firstpatentassignments, how= 'inner')

In [27]:
crosswalk_final['name_0'] = crosswalk_final['name_0'].astype(str)
crosswalk_2['name_0'] = crosswalk_2['name_0'].astype(str)

In [28]:
crosswalk_2 = crosswalk_2[~crosswalk_2['name_0'].isin(np.unique(crosswalk_final['name_0'].values))]

In [29]:
crosswalk_2 = crosswalk_2[~crosswalk_2['permno'].isin(np.unique(crosswalk_final['permno'].values))]

In [30]:
crosswalk_2 = crosswalk_2[['name_0', 'permno', 'recorded_date', 'patent_ids']]

In [31]:
crosswalk_2 = crosswalk_2.drop_duplicates(['name_0', 'permno'])

In [32]:
#resolve one company name but multiple permnos
#drop all entries
crosswalk_2 = crosswalk_2.drop_duplicates(['name_0'], keep = False)
duplicated_companynames = np.unique(crosswalk_2['name_0'][crosswalk_2['name_0'].duplicated()].values)

In [33]:
#resolve one permno but multiple company names
duplicated_permnos = np.unique(crosswalk_2['permno'][crosswalk_2['permno'].duplicated()].values)
crosswalk_2_one_on = crosswalk_2[~crosswalk_2['permno'].isin(duplicated_permnos)]
crosswalk_2_multiple_on_one =crosswalk_2[crosswalk_2['permno'].isin(duplicated_permnos)]

In [34]:
#resolve one company name but multiple permnos
#drop the cases due to multiple patent ids 
crosswalk_final_2 = pd.DataFrame()
for i in duplicated_permnos:
    temp = crosswalk_2_multiple_on_one[crosswalk_2_multiple_on_one['permno'] == i]
    temp = temp.drop_duplicates(subset = ['name_0'], keep = 'first')
    try:
        matches = match_strings(temp['name_0'])
    except ValueError:
        print("only stop words")
        continue
    matches = matches[matches['left_side'] != matches['right_side']]
    if matches.empty:
        continue
    name = matches['left_side'].value_counts().keys()[0]
    crosswalk_final_2 = crosswalk_final_2.append(temp[temp['name_0'] == name])

In [35]:
crosswalk_2 = crosswalk_2_one_on.append(crosswalk_final_2)

In [36]:
#Check final crosswalk
crosswalk_final = pd.concat([crosswalk_final, crosswalk_2])
crosswalk_final[crosswalk_final['permno'].duplicated(keep = False)].sort_values(['permno'])

,name_0,permno,recorded_date,patent_ids
970918,3 m innovative,22592.0,20040130,7295654
0,3m innovative,22592.0,19990401,6428933
329213,marathon ashland petroleum,24272.0,19980406,5961709
5,ashland licensing intellectual property,24272.0,20050817,6342543
141736,regalbeloit,58771.0,19950703,5476164
42,regal beloit,58771.0,20061204,7482770
803164,advanced microdevices,61241.0,20011231,6970758
1,advanced micro devices,61241.0,19930330,5381037
1193277,boston scientific scimed,77605.0,20050108,7776033
6,boston scientific,77605.0,19930607,5401257


In [42]:
#old records are not kept
#check these companies indeed have multipe permnos
crosswalk_final[crosswalk_final['name_0'].duplicated(keep = False)].sort_values(['name_0'])


,name_0,permno,recorded_date,patent_ids
931871,advanced micro devices,91029.0,20010313,7217652
1,advanced micro devices,61241.0,19930330,5381037
27146,ashland licensing intellectual property,67361.0,20050817,4588637
5,ashland licensing intellectual property,24272.0,20050817,6342543
95826,boston scientific,77380.0,20061031,5314407
6,boston scientific,77605.0,19930607,5401257
51953,electronics imaging,78589.0,19991216,4917488
11,electronics imaging,77971.0,19940202,5543940
21,hei,32678.0,20050214,7334984
20,hei,40540.0,19951013,5847930


In [46]:
crosswalk_final.loc[21, 'name_0'] = 'heico'

'hei'

In [47]:
crosswalk_final = crosswalk_final.drop([30930])
crosswalk_final = crosswalk_final.drop([931871])
crosswalk_final = crosswalk_final.drop([95826])
crosswalk_final = crosswalk_final.drop([30])
crosswalk_final = crosswalk_final.drop([89259])

KeyError: '[30930] not found in axis'

In [52]:
crosswalk_final.to_csv("permno_processedfirstassignorname_crosswalk.csv", index = False)